In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import datetime
import matplotlib.pyplot as plt
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_squared_error, r2_score


In [2]:
dataFull = pd.read_csv("data/NE_LINCOLN_2008_2024.csv")
dataNoNa = pd.read_csv("data/NE_LINCOLN_NO_NA.csv")

In [3]:
# dropping columns not used for VAR
dataFull.drop(columns = ["Unnamed: 0", "id", "location", "doseEquivalent", "status", "gammaSum"], axis = 1, inplace = True)
dataNoNa.drop(columns = ["Unnamed: 0", "id", "location", "doseEquivalent", "status", "gammaSum"], axis = 1, inplace = True)

In [4]:
# augmented dickey-fuller test to make sure data is stationary using 
# https://www.analyticsvidhya.com/blog/2021/08/vector-autoregressive-model-in-python/
def adfTest(series, title = ""):
    print(f'Augmented Dickey-Fuller Test: {title}')
    results = adfuller(series.dropna(), autolag = "AIC")
    labels = ['ADF test statistic', 'p-value', '# lags used', '# observations']
    out = pd.Series(results[0:4], index = labels)
    for key, val in results[4].items():
        out[f'critical value({key})']=val
    print(out.to_string())
    if results[1] <= 0.05:
        #print("Strong evidence against the null hypothesis")
        #print("Reject the null hypothesis")
        print("Data has no unit root and is stationary")
    else:
        #print("Weak evidence against the null hypothesis")
        #print("Fail to reject the null hypothesis")
        print("Data has a unit root and is non-stationary")

In [5]:
# testing if the data is stationary
# since dataNoNa is subset of dataFull, only dataFull is tested
# do not run, it takes about 5 minutes...
for i in range(2, 10):
    adfTest(dataFull['gamma' + str(i)], i)

Augmented Dickey-Fuller Test: 2
ADF test statistic    -7.740945e+00
p-value                1.062356e-11
# lags used            6.900000e+01
# observations         1.220110e+05
critical value(1%)    -3.430404e+00
critical value(5%)    -2.861564e+00
critical value(10%)   -2.566783e+00
Data has no unit root and is stationary
Augmented Dickey-Fuller Test: 3
ADF test statistic    -1.444869e+01
p-value                7.158214e-27
# lags used            7.100000e+01
# observations         1.219930e+05
critical value(1%)    -3.430404e+00
critical value(5%)    -2.861564e+00
critical value(10%)   -2.566783e+00
Data has no unit root and is stationary
Augmented Dickey-Fuller Test: 4
ADF test statistic    -1.699695e+01
p-value                8.785375e-30
# lags used            7.100000e+01
# observations         1.219910e+05
critical value(1%)    -3.430404e+00
critical value(5%)    -2.861564e+00
critical value(10%)   -2.566783e+00
Data has no unit root and is stationary
Augmented Dickey-Fuller Test

In [5]:
# simple data conversions to usable types
dataFull.dropna(inplace = True)
dataFull["time"] = pd.DatetimeIndex(pd.to_datetime(dataFull["time"]))
entries2024 = int(sum(dataFull[dataFull['time'].dt.year == 2024].count()) / 9)
dataFull['time'] = pd.to_numeric(dataFull['time'])
for i in range(2, 10):
    dataFull["gamma" + str(i)] = dataFull['gamma' + str(i)].astype(float)

In [8]:
# counting number of records from 2024 to make the test group
train = dataFull[:-entries2024]
test = dataFull[-entries2024:]


In [9]:
train = np.asarray(train)
test = np.asarray(test)
for i in range(1, 9):
    train[:, i] = train[:, i].astype(np.float64)
    test[:, i] = test[:, i].astype(np.float64)

In [9]:
for i in range(1, 11):
    model = VAR(np.asarray(train))
    results = model.fit(i)
    print("Order: " + str(i))
    print("AIC: " + str(results.aic))
    print("BIC: " + str(results.bic))

Order: 1
AIC: 112.94643677430228
BIC: 112.95400137091302
Order: 2
AIC: 112.9464380644538
BIC: 112.9608109102164
Order: 3
AIC: 112.9473472795681
BIC: 112.96852848078207
Order: 4
AIC: 112.94824271064698
BIC: 112.97623237361447
Order: 5
AIC: 112.94914025082791
BIC: 112.98393848185366
Order: 6
AIC: 112.94996272526201
BIC: 112.99156963065343
Order: 7
AIC: 112.95079419924801
BIC: 112.99920988531508
Order: 8
AIC: 112.95168861479448
BIC: 113.0069131878498
Order: 9
AIC: 112.95254259729887
BIC: 113.01457616365768
Order: 10
AIC: 112.95348810083856
BIC: 113.0223307668187


In [10]:
model = VAR(np.asarray(train))
results = model.fit(1)

In [11]:
pred = results.forecast(np.asarray(test), steps = entries2024)
pred[0:5,]

array([[1.73569192e+18, 2.70201358e+03, 1.56517724e+03, 4.49723498e+02,
        2.56703149e+02, 1.55527957e+02, 1.95088411e+02, 1.11719194e+02,
        3.64116613e+01],
       [1.73569706e+18, 2.70202159e+03, 1.56518187e+03, 4.49724830e+02,
        2.56703910e+02, 1.55528418e+02, 1.95088989e+02, 1.11719524e+02,
        3.64117692e+01],
       [1.73570220e+18, 2.70202959e+03, 1.56518651e+03, 4.49726162e+02,
        2.56704670e+02, 1.55528879e+02, 1.95089567e+02, 1.11719855e+02,
        3.64118770e+01],
       [1.73570735e+18, 2.70203760e+03, 1.56519115e+03, 4.49727494e+02,
        2.56705430e+02, 1.55529339e+02, 1.95090145e+02, 1.11720186e+02,
        3.64119849e+01],
       [1.73571249e+18, 2.70204560e+03, 1.56519578e+03, 4.49728826e+02,
        2.56706191e+02, 1.55529800e+02, 1.95090723e+02, 1.11720517e+02,
        3.64120927e+01]])

In [14]:
var_mse = mean_squared_error(test, pred)
var_r2 = r2_score(test, pred)

print(f'MSE: {var_mse}')
print(f'R2: {var_r2}')

MSE: 1.314163274671803e+32
R2: -4.446750062499164
